In [1]:
import nixio
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

In [2]:
_nix_dir = join("/", "mnt", "c", "Users", "conta", "git", "USZ_NCH", "Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
                "data_nix")
_file_path = join(_nix_dir, "Data_Subject_01_Session_01.h5")
nixfile = nixio.File.open(_file_path, nixio.FileMode.ReadOnly)
# nixfile.pprint()

_general = nixfile.sections["General"]
recording_location = _general.props["Recording location"].values[0]
recording_setup = _general.sections["Recording setup"].props["Recording setup EEG"].values[0]

_electrode_map = nixfile.blocks[
    "Data_Subject_01_Session_01"].groups[
    "Scalp EEG electrode information"].data_arrays[
    "Scalp_Electrode_EEGLAB_BESA_Coordinates"]

# See <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
# in x, y ,z
eeg_electrode_locations = np.ndarray(_electrode_map.shape)
_electrode_map.read_direct(eeg_electrode_locations)

In [3]:
print(recording_location)
print(recording_setup)
#print(eeg_electrode_locations)
print(_general.sections["Related publications"].props)

Schweizerische Epilepsie-Zentrum, 8008 Zurich, Switzerland
NicOne; original sampling rate, 256 Hz; passband, 0.3 to 100 Hz
[Property: {name = Publication name}, Property: {name = Publication DOI}]


In [4]:
# BESA map: +X is nose, +Y is left, +Z is up according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
eeg_electrode_locations_df = pd.DataFrame(eeg_electrode_locations, columns=['X', 'Y', 'Z'])
# By eye: ["F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2", "F7", "F8", "T7", "T8", "P7", "P8", "FZ", "CZ", "PZ", "LPA", "RPA"]
_labels = nixfile.blocks["Data_Subject_01_Session_01"].groups["Scalp EEG data"].data_arrays[
    "Scalp_EEG_Data_Trial_01"].dimensions[0].labels
eeg_electrode_locations_df.insert(3, "location", _labels)
print(eeg_electrode_locations_df.to_string())
fig = px.scatter_3d(eeg_electrode_locations_df, x='X', y='Y', z='Z', color=df.index)
fig.show()

               X             Y          Z location
0   5.755106e+01  4.820043e+01  39.869712       F3
1   5.758403e+01 -4.814260e+01  39.891983       F4
2   3.868125e-15  6.317128e+01  56.871691       C3
3   3.867882e-15 -6.316731e+01  56.876102       C4
4  -5.755106e+01  4.820043e+01  39.869712       P3
5  -5.758403e+01 -4.814260e+01  39.891983       P4
6  -8.078401e+01  2.613301e+01  -4.001085       O1
7  -8.078401e+01 -2.613301e+01  -4.001085       O2
8   4.987138e+01  6.842334e+01  -7.489518       F7
9   4.992653e+01 -6.838359e+01  -7.485085       F8
10  5.176493e-15  8.453854e+01  -8.845083       T3
11  5.176493e-15 -8.453854e+01  -8.845083       T4
12 -4.987138e+01  6.842334e+01  -7.489518       T5
13 -4.992653e+01 -6.838359e+01  -7.485085       T6
14  6.073848e+01 -0.000000e+00  59.462904       Fz
15  5.204749e-15 -0.000000e+00  85.000000       Cz
16 -6.073848e+01 -7.438319e-15  59.462904       Pz
17  3.680313e-15  6.010408e+01 -60.104076       A1
18  3.680313e-15 -6.010408e+01 

NameError: name 'df' is not defined

In [ ]:
_sampled_dimension = nixfile.blocks["Data_Subject_01_Session_01"].groups["Scalp EEG data"].data_arrays[
    "Scalp_EEG_Data_Trial_01"].dimensions[1]
print(f"Start at {_sampled_dimension.offset} {_sampled_dimension.unit} w.r.t. fixation")
print(f"Sampled every {_sampled_dimension.sampling_interval} {_sampled_dimension.unit}")

_events = nixfile.blocks["Data_Subject_01_Session_01"].groups["Trial events single tags scalp EEG"].tags
_EVENTS_PER_TRIAL = 5
trials = 1
for i in range(0, trials * _EVENTS_PER_TRIAL, _EVENTS_PER_TRIAL):
    print("--")
    for j in range(_EVENTS_PER_TRIAL):
        _event = _events[i + j]
        print(
            f"Event {_event.name}: {_event.position[1]} {_event.units[0]} (w.r.t. fixation), lasted {_event.extent[1]} {_event.units[0]}")


In [ ]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups["Scalp EEG data"].data_arrays[
          "Scalp_EEG_Data_Trial_01"].dimensions[1])

In [ ]:
ref = nixfile.blocks["Data_Subject_01_Session_01"].groups["Scalp EEG data"].data_arrays[
    "Scalp_EEG_Data_Trial_01"].dimensions[0].labels
print(ref)
for i in range(1, 50):
    assert nixfile.blocks["Data_Subject_01_Session_01"].groups["Scalp EEG data"].data_arrays[
               f"Scalp_EEG_Data_Trial_{i:02d}"].dimensions[0].labels == ref
print("All trials have the same electrode order")

In [ ]:
ieeg_electrode_information = nixfile.blocks["Data_Subject_01_Session_01"].groups[
    "iEEG electrode information"].data_arrays
print(ieeg_electrode_information)
print(ieeg_electrode_information["iEEG_Electrode_Map"][:])
print(ieeg_electrode_information["iEEG_Electrode_Map"].sources["iEEG electrode 5"].sources)
print(ieeg_electrode_information["iEEG_Electrode_MNI_Coordinates"].shape)
print(ieeg_electrode_information["iEEG_Electrode_Manual_Entry"].dimensions[0].labels)
print(list(zip(ieeg_electrode_information["iEEG_Electrode_Map"][:],
               ieeg_electrode_information["iEEG_Electrode_Manual_Entry"].dimensions[0].labels)))

In [ ]:
print(nixfile.blocks["Data_Subject_01_Session_01"].groups["Spike times"].data_arrays)

In [ ]:
print(nixfile.sections["Session"].sections["Trial properties"].sections)

In [ ]:
print(nixfile.sections["Subject"].props["Age"].name)
print(nixfile.sections["Subject"].props["Age"].values[0])